Libraries

In [5]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import os
import polars as pl
import pandas as pd
df = pd.read_csv("test_file.csv", delimiter='\t')
df = df.sort_values(by=['PRODUCT NAME'], ascending=True)
df

,GENE ID,LOCUS TAG,PRODUCT NAME,SCAFFOLD ID,GENE SEQ LENGTH,READS COUNT,COVERAGE,MEDIAN,READS COUNT ANTISENSE,COVERAGE ANTISENSE,MEDIAN ANTISENSE
3312,3300002862 assembled Ga0004695J43176_1067321,Ga0004695J43176_1067321,(acyl-carrier-protein) S-malonyltransferase,Ga0004695J43176_1067321,165,3,2.376879e-08,3,5,3.961465e-08,3
6814,3300002862 assembled Ga0004695J43176_1024641,Ga0004695J43176_1024641,"1,4-alpha-glucan branching enzyme",Ga0004695J43176_1024641,420,29,9.026482e-08,7,33,1.027151e-07,11
991,3300002862 assembled Ga0004695J43176_1070911,Ga0004695J43176_1070911,1-acyl-sn-glycerol-3-phosphate acyltransferase,Ga0004695J43176_1070911,185,4,2.826559e-08,3,4,2.826559e-08,3
4200,3300002862 assembled Ga0004695J43176_1069501,Ga0004695J43176_1069501,16S rRNA uridine-516 pseudouridylate synthase ...,Ga0004695J43176_1069501,176,5,3.713874e-08,5,5,3.713874e-08,3
4003,3300002862 assembled Ga0004695J43176_1020621,Ga0004695J43176_1020621,2-keto-3-deoxygluconate permease,Ga0004695J43176_1020621,681,36,6.910750e-08,8,46,8.830403e-08,9
...,...,...,...,...,...,...,...,...,...,...,...
2240,3300002862 assembled Ga0004695J43176_1042861,Ga0004695J43176_1042861,picornavirus capsid protein,Ga0004695J43176_1042861,282,15,6.953636e-08,8,14,6.490060e-08,7
3072,3300002862 assembled Ga0004695J43176_1065931,Ga0004695J43176_1065931,tRNA splicing endonuclease,Ga0004695J43176_1065931,213,5,3.068741e-08,4,4,2.454993e-08,3
595,3300002862 assembled Ga0004695J43176_1032791,Ga0004695J43176_1032791,tRNA synthetases class II (A),Ga0004695J43176_1032791,1782,76,5.575396e-08,6,79,5.795477e-08,6
8595,3300002862 assembled Ga0004695J43176_1046031,Ga0004695J43176_1046031,tRNA synthetases class II core domain (F),Ga0004695J43176_1046031,321,10,4.072534e-08,5,18,7.330562e-08,6


File Enumeration and filtering:

In [81]:
def filter_and_append_to_csv(input_file, output_file, product):
    try:
        df_input = pd.read_csv(input_file, on_bad_lines='skip')
        
        print("First few rows of the input DataFrame:")
        display(df_input.head())
        
        filtered_df = df_input[df_input.iloc[:, 0].str.contains(product, na=False)]
        
        print(f"Number of rows in filtered DataFrame: {len(filtered_df)}")
        
        # Write to output file, append if exists
        if not os.path.exists(output_file):
            filtered_df.to_csv(output_file, index=False)
        else:
            filtered_df.to_csv(output_file, mode='a', header=False, index=False)

        return filtered_df

    except Exception as e:
        print(f"Error processing {input_file}: {str(e)}")
        return None

def enum():
    folder_path = os.getcwd()
    print("Enumerating files...")
    
    csv_files = [file for file in os.listdir(folder_path) if file.startswith('rnaseq_data')]
    
    renamed_files = []
    # Rename each file with a numbered format
    for index, file in enumerate(csv_files, start=1):
        old_path = os.path.join(folder_path, file)
        new_name = f'{index}.csv'
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)
        renamed_files.append(new_name)
    
    print("Files have been successfully renamed.")
    return renamed_files


testing enum:

In [82]:
renamed_files = enum()
print("Renamed files:", renamed_files)

Enumerating files...
Files have been successfully renamed.
Renamed files: []


Testing filter_append_to_Csv:

In [83]:
if renamed_files:
    test_input_file = renamed_files[0]  # Use the first renamed file
    test_output_file = "output_file.csv"
    test_product = "methanol dehydrogenase (cytochrome c) subunit 1"  # Change this to a product you want to filter by

    filtered_df = filter_and_append_to_csv(test_input_file, test_output_file, test_product)

    if filtered_df is not None and not filtered_df.empty:
        display(filtered_df)
    else:
        print("No data to display.")
else:
    print("No files were renamed.")

No files were renamed.


Linkgen function: 

In [84]:
def linkgen():
    try:
        filename = "output_file.csv"
        df = pl.read_csv(filename)
        df = df.with_columns(pl.col("Gene seq length").str.concat([
            "https://img.jgi.doe.gov/cgi-bin/mer/main.cgi?section=MetaGeneDetail&page=genePageMainFaa&taxon_oid=", 
            df["Gene seq length"], 
            "&data_type=assembled&gene_oid=", 
            df["Gene seq length"].str.split(",").arr.first()
        ], sep='').alias("Link"))

        df.select(["Gene seq length", "Link"]).write_csv("Link_Output.csv")
        print("Done generating links.")
    except Exception as e:
        print(f"Error generating links: {str(e)}")

In [85]:
linkgen()

Error generating links: Gene seq length
